In [1]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import pandas as pd
import os
from backend.domain.model import Company, Filing

c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uow_instance = uow.UnitOfWork()
company = service.get_company_by_ticker('GOOG', uow_instance)


In [3]:
combined = service.get_consolidated_income_statements('GOOG', uow_instance, '10-K')

Years covered in last filing (2025-02-05):
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
loading filings for [<backend.domain.model.Filing object at 0x0000026009E5FD90>, <backend.domain.model.Filing object at 0x0000026009E5F1D0>]
loading filings for 2024-12-31 10-K
loading filings for 2021-12-31 10-K
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
Index(['2022-01-01:2022-12-31', '2023-01-01:2023-12-31',
       '2024-01-01:2024-12-31'],
      dtype='object', name='date_range')
Index(['2019-01-01:2019-12-31', '2020-01-01:2020-12-31',
       '2021-01-01:2021-12-31'],

c:\Users\Site Oracle\reduceComplexity\backend\domain\model.py:594: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  self.df = self.df.applymap(self.convert_to_millions)


In [4]:
def remove_zero_decimals(val):
    try:
        num = float(val)
        if num == int(num):
            return int(num)
        else:
            return num
    except (ValueError, TypeError):
        return val

In [5]:
import pandas as pd
pd.options.display.float_format = '{:.10g}'.format
combined.df = combined.df.map(remove_zero_decimals)

In [6]:
combined.df

date_range,2019-01-01:2019-12-31,2020-01-01:2020-12-31,2021-01-01:2021-12-31,2022-01-01:2022-12-31,2023-01-01:2023-12-31,2024-01-01:2024-12-31
Revenue,161857,182527,257637,282836,307394,350018
Cost of Revenue,71896,84732,110939,126203,133332,146306
R&D Expense,26018,27573,31562,39500,45427,49326
Sales & Marketing Expense,18464,17946,22912,26567,27917,27808
G&A Expense,9551,11052,13510,15724,16425,14188
Costs & Expenses,127626,141303,178923,207994,223101,237628
Operating Income,34231,41224,78714,74842,84293,112390
Non-operating Income,5394,6858,12020,-3514,1424,7425
Pre-tax Income,39625,48082,90734,71328,85717,119815
Income Tax Expense,5282,7813,14701,11356,11922,19697


In [7]:
def convert_to_millions(val):
    try:
        num = float(val)
        
        if abs(num) >= 100_000:
            result = round(num / 1000000, 2)
            # Check if the result has no meaningful decimal part
            if result == int(result):
                return int(result)
            else:
                return result
        else:
            return num
    except (ValueError, TypeError):
        return val

In [8]:
def is_sparse_row(row):
    # Count zero values in the row
    numeric_values = 0
    zero_values = 0
    
    for val in row:
        try:
            num = float(val)
            numeric_values += 1
            if num == 0:
                zero_values += 1
        except (ValueError, TypeError):
            # Skip non-numeric values
            pass
    
    # If no numeric values, don't delete
    if numeric_values == 0:
        return False
        
    # Calculate percentage of zeros
    zero_percentage = zero_values / numeric_values if numeric_values > 0 else 0
    
    # Return True if more than 50% zeros
    return zero_percentage > 0.5

In [9]:
# combined.df = combined.df.applymap(convert_to_millions)


In [10]:
# combined.df = combined.df.loc[[not is_sparse_row(row) for _, row in combined.df.iterrows()]]

In [11]:
combined.df

date_range,2019-01-01:2019-12-31,2020-01-01:2020-12-31,2021-01-01:2021-12-31,2022-01-01:2022-12-31,2023-01-01:2023-12-31,2024-01-01:2024-12-31
Revenue,161857,182527,257637,282836,307394,350018
Cost of Revenue,71896,84732,110939,126203,133332,146306
R&D Expense,26018,27573,31562,39500,45427,49326
Sales & Marketing Expense,18464,17946,22912,26567,27917,27808
G&A Expense,9551,11052,13510,15724,16425,14188
Costs & Expenses,127626,141303,178923,207994,223101,237628
Operating Income,34231,41224,78714,74842,84293,112390
Non-operating Income,5394,6858,12020,-3514,1424,7425
Pre-tax Income,39625,48082,90734,71328,85717,119815
Income Tax Expense,5282,7813,14701,11356,11922,19697


In [12]:
combined.analyze_period_coverage()

{'coverage_by_year': {2019: ['Q1'],
  2020: ['Q1'],
  2021: ['Q1'],
  2022: ['Q1'],
  2023: ['Q1'],
  2024: ['Q1']},
 'missing_periods': {2019: ['Q2', 'Q3', 'Q4'],
  2020: ['Q2', 'Q3', 'Q4'],
  2021: ['Q2', 'Q3', 'Q4'],
  2022: ['Q2', 'Q3', 'Q4'],
  2023: ['Q2', 'Q3', 'Q4'],
  2024: ['Q2', 'Q3', 'Q4']},
 'has_continuous_coverage': True,
 'years_analyzed': [2019, 2020, 2021, 2022, 2023, 2024]}

In [13]:
for filing in combined.source_filings:
    try: 
        print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus)
    except:
        print(filing.form)

FY 2024
FY 2021


In [14]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

['2019-01-01:2019-12-31', '2020-01-01:2020-12-31', '2021-01-01:2021-12-31', '2022-01-01:2022-12-31', '2023-01-01:2023-12-31', '2024-01-01:2024-12-31']


In [15]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


Loaded data for filing date 2025-04-25, form 10-Q


In [16]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

AttributeError: 'NoneType' object has no attribute 'table'

In [31]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')

Loaded data for filing date 2024-12-06, form 10-Q
Loaded data for filing date 2024-09-06, form 10-Q
Loaded data for filing date 2024-06-07, form 10-Q
Loaded data for filing date 2022-12-02, form 10-Q
Loaded data for filing date 2022-09-02, form 10-Q
Loaded data for filing date 2022-06-02, form 10-Q
Loaded data for filing date 2020-12-04, form 10-Q
Loaded data for filing date 2020-09-04, form 10-Q
Loaded data for filing date 2020-06-11, form 10-Q
Loaded data for filing date 2018-12-07, form 10-Q
Loaded data for filing date 2018-09-06, form 10-Q
Loaded data for filing date 2018-06-07, form 10-Q
Loaded data for filing date 2016-12-06, form 10-Q
Loaded data for filing date 2016-09-01, form 10-Q
Loaded data for filing date 2016-06-02, form 10-Q
Loaded data for filing date 2014-12-09, form 10-Q
Loaded data for filing date 2014-09-05, form 10-Q
Loaded data for filing date 2014-06-06, form 10-Q
Loaded data for filing date 2012-12-05, form 10-Q
Loaded data for filing date 2012-09-05, form 10-Q
